# Constituent Data Pipeline

This app aggregates constituent data to produce one file containing data about each constituent and another containing the number of new acquisitions per calendar day.

## Instructions:
- Ensure that your working directory contains the files "cons.csv", "cons_email.csv", and "cons_email_chapter_subscription.csv"
- If you wish to apply filters to the data, change the appropriate setting in the Settings cell.
- Run the Settings cell.
- Run the Pipeline cell.
- After execution, look for the files "people.csv" and "acquisition_facts.csv" in your working directory.

## Settings
- **Note on "source_col" setting:**   
There are two fields in the constituents table which might qualify for the "code" column: "source" and "subsource", necessitating this setting.   
The default setting ("both") means that in cases where both source and subsource are available, the "code" column will contain source and subsource, separated by a hyphen(-).
Replacing "both" with "source" or "subsource" will pull data only from that column, ignoring the other.  


- The other settings are filters--refer to the comments above each setting to learn more.

In [246]:
# SOURCE_COL
# Column from which to pull source code information. Options are "source", "subsource", or "both"
source_col = "both"

# FILTER UNVALIDATED CONSTITUENTS
# If True, only include people who are validated
filter_unvalidated = False

# FILTER BANNED CONSTITUENTS
# If True, only include people who are not banned
filter_banned = False

# FILTER CONSTITUENT STATUS
# If True, only include people whose status is 1 (do not include people with status == 0)
filter_cons_status = False

# FILTER CONSTITUENTS WITH NO EMAIL
# If True, only include people who are connected with an email
filter_no_email = False

# FILTER EMAIL STATUS
# If True, only include people whose primary email address has a status of 1 (as opposed to 0)
filter_email_status = False

## Pipeline
- Please do not make any changes to this cell.
- Please allow a few minutes for execution.  
- After execution, "people.csv" and "acquisition_facts.csv" should appear in your working directory
- Note: This might produce a "FutureWarning: elementwise comparison failed";  
This is because of a numpy bug, and should not affect the end result: click [here](https://stackoverflow.com/questions/40659212/futurewarning-elementwise-comparison-failed-returning-scalar-but-in-the-futur) to learn more.

In [247]:
import pandas as pd
import numpy as np


def get_source_code(row: pd.Series):
    sources = []
    
    if pd.notnull(row.loc['source']) and source_col in ['both', 'source']:
        sources.append(row.loc['source'])
    
    if pd.notnull(row.loc['subsource']) and source_col in ['both', 'subsource']:
        sources.append(row.loc['subsource'])
        
    if sources:
        return '-'.join(sources)
    else:
        return np.nan


# Extract Constituents
constituents = pd.read_csv('cons.csv', index_col = 'cons_id', usecols=['cons_id', 'source', 'subsource', 'is_validated',
                                                                       'is_banned', 'create_dt', 'modified_dt', 'status'])
# Filter out unvalidated constituents
if filter_unvalidated:
    constituents = constituents[constituents.is_validated == 1]

# Filter out banned constituents
if filter_banned:
    constituents = constituents[constituents.is_banned == 0]
    
# Filter out constituents with a status of 0
if filter_cons_status:
    constituents = constituents[constituents.status == 1]
    
constituents.drop(columns = ['is_validated', 'is_banned', 'status'], inplace=True)


# Extract Emails
emails = pd.read_csv('cons_email.csv', index_col='cons_email_id', usecols=['cons_email_id', 'cons_id', 'is_primary',
                                                                           'email', 'status'])
# Only keep primary emails
emails = emails[emails.is_primary == 1]

# Filter emails with a status of 0
if filter_email_status:
    emails = emails[emails.status == 1]

emails.drop(columns=['is_primary', 'status'], inplace=True)


# Extract Subscriptions
subscriptions = pd.read_csv('cons_email_chapter_subscription.csv', index_col = 'cons_email_chapter_subscription_id')
# Only keep chapter 1 subscriptions
subscriptions = subscriptions[subscriptions.chapter_id == 1]
subscriptions.drop(columns=['unsub_dt', 'modified_dt', 'chapter_id'], inplace=True)


# Merge DataFrames
email_sub = emails.merge(subscriptions, on='cons_email_id', how='left').set_index('cons_email_id')
email_sub['isunsub'].fillna(0, inplace=True)

# Filter constituents with no email
merge_method = 'inner' if filter_no_email else 'left'

people = constituents.merge(email_sub, on='cons_id', how=merge_method).set_index('cons_id')

# Construct appropriate "code" column
people['code'] = people.apply(get_source_code, axis=1)

# Re-order and rename columns as per client specifications
people = people[['email', 'code', 'isunsub', 'create_dt', 'modified_dt']]
people.rename(columns={'isunsub': 'is_unsub', 'create_dt': 'created_dt', 'modified_dt': 'updated_dt'}, inplace=True)

# Load constituent data into "people.csv"
people.to_csv('people.csv')

# Calculate number of acquisitions per calendar day
dates = pd.to_datetime(people.created_dt).dt.date
acquisition_facts = dates.value_counts().rename_axis('acquisition_date').reset_index(name='acquisitions')
acquisition_facts.sort_values('acquisition_date', inplace=True)

# Load acquisition frequency data into "acquisition_facts.csv"
acquisition_facts.to_csv('acquisition_facts.csv', index=False)

C:\Users\CISS\anaconda3\envs\bee2a_khayaleya\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
